In [1]:
import sqlite3

In [6]:
db = './scrapedata.db'
con = sqlite3.connect(db)
cur = con.cursor()
for d in ["nytimes", "bonappetit", "epicurious", "food52", "seriouseats", "liquor", "allrecipes", "smittenkitchen"]:
    cur.execute("SELECT Count() FROM recipes WHERE url LIKE ? ", ('%'+d+'%',))
    print(d, ':', cur.fetchone()[0])
print("total : ", cur.execute("SELECT Count() FROM recipes").fetchone()[0])


nytimes : 18444
bonappetit : 6028
epicurious : 16901
food52 : 16071
seriouseats : 1999
liquor : 610
allrecipes : 15173
smittenkitchen : 359
total :  75580


In [ ]:
cur.execute("SELECT ingredients FROM recipes WHERE url LIKE '%smitten%bolognese%' ").fetchall()
# con.commit()

[('1 small yellow onion,1 slim carrot,1 rib celery,2 cloves garlic (optional),2 ounces pancetta, diced (optional) or 2 to 3 tablespoons olive oil,Kosher salt,Freshly ground black pepper and red pepper flakes,1 pound ground beef (80/20 or 85/15),1/2 cup whole milk, or 3/4 cup of a low-fat one,1/2 cup dry white wine,1 6-ounce can tomato paste,Water,1 bay leaf,A few gratings of fresh nutmeg,1 pound dried tagliatelle',),
 ('1 medium onion, coarsely chopped (1-inch pieces are fine),1 large or 2 slim carrots, coarsely chopped,2 ribs celery, coarsely chopped,3 cloves garlic, coarsely chopped,2 to 3 tablespoons olive oil,Kosher salt,Freshly ground black pepper,2 pounds ground beef or, if you prefer, 1 pound each beef and pork,1 1/4 cups tomato paste (from 2 6-ounce cans),2 cups red wine, preferably hearty but really, anything you like to drink,Water as needed,2 bay leaves,A few sprigs thyme, tied in a bundle,1 1/2 cups all-purpose flour,2 large eggs,1/2 teaspoon fine sea or table salt,1 to 2 t

In [80]:
cur.execute("SELECT title, img, url FROM recipes WHERE ingredients MATCH '%mayo% AND "
 "(mayo)' ORDER BY bm25(recipes)").fetchall()
# con.commit()

OperationalError: fts5: syntax error near "%"

In [8]:
db = './test.db'
con = sqlite3.connect(db)
cur = con.cursor()
cur.execute("CREATE VIRTUAL TABLE IF NOT EXISTS recipes USING FTS5(title, img, url, ingredients, tokenize='trigram')")
db2 = './scrapedata.db'
con2 = sqlite3.connect(db2)
cur2 = con2.cursor()

In [12]:
cur.execute("SELECT * from recipes").fetchone()

In [13]:
for i in cur2.execute("SELECT * from recipes").fetchall():
    cur.execute("INSERT INTO recipes VALUES (?, ?, ?, ?)", 
            (i[0],i[1],i[2],i[3]))

In [15]:
con.commit()
con.close()

In [36]:
def search(ingredient_list, must_have_list):
    query = "SELECT title, img, url FROM recipes "
    if ingredient_list != [] or must_have_list != []:
        query += "WHERE ingredients MATCH '("
        if ingredient_list != []:
            query += " OR ".join(ingredient_list) + ")"
            if must_have_list != []:
                query += " AND ("
            else:
                query += "'"
        query += " AND ".join(must_have_list) 
        if must_have_list != []:
            query += ")'"
        query += " ORDER BY bm25(recipes)"
    print(query)
    return cur.execute(query).fetchall()

In [39]:
search([],['crush red pepper'])

SELECT title, img, url FROM recipes WHERE ingredients MATCH '(crush red pepper)' ORDER BY bm25(recipes)


[('Shrimp Fra Diavolo',
  'https://www.allrecipes.com/img/misc/og-default.png',
  'https://www.allrecipes.com/recipe/11731/shrimp-fra-diavolo/'),
 ('Marinated Olives with Tangerine and Rosemary',
  '',
  'https://www.epicurious.com/recipes/food/views/marinated-olives-with-tangerine-and-rosemary-236245'),
 ('Boiled Shrimp Recipe',
  'https://static01.nyt.com/applications/cooking/f5c4090/assets/NYTCookingLogo.png',
  'https://cooking.nytimes.com/recipes/2465-boiled-shrimp'),
 ('Thick Tomato Sauce Recipe',
  'https://static01.nyt.com/applications/cooking/f5c4090/assets/NYTCookingLogo.png',
  'https://cooking.nytimes.com/recipes/6688-thick-tomato-sauce'),
 ('Spicy Garlic Chickpeas',
  'https://assets.bonappetit.com/photos/57ad50ce53e63daf11a4dde7/16:9/w_1280,c_limit/spicy-garlic-chickpeas.jpg',
  'https://www.bonappetit.com/recipe/spicy-garlic-chickpeas'),
 ('Summer Corn Pizza Recipe on Food52',
  'https://images.food52.com/hcK8Y7wWUYMvcc7htSfrRHWnapM=/fit-in/1200x1200/c947f605-2d61-4a10-9